In [63]:
from urllib.parse import urlencode
from urllib.request import urlopen
from urllib.request import urlretrieve
import re
import requests
import os
import pandas as pd

In [28]:
class AzLyrics:
    '''
    lyrics downloader from AZLyrics.com
    '''
    
    def __init__(self, url):
        self.publc_url = url
        
    def url_raw(self, pub_url):
    temp_file, _ = urlretrieve(self.publc_url)
    with open(temp_file) as file:
        return file.read()
        
        
    def get_album_info(url_out_raw):
        header = re.findall(r'<h1><strong>(.*?) Lyrics</strong></h1>', url_out_raw)[0].strip()


        a_dict = {}
        a_dict['band_name'] = header
        try:
            related_artists_slab = re.findall(r'Related artists:<br>\n(.*?)\n\n',url_out_raw, re.DOTALL)[0].strip()
            related_artists = re.findall(r'\">(.*?)</a>', related_artists_slab)
            a_dict['related_artists'] = related_artists
        except:
            pass
        a_dict['albums'] = {}

        albums_info = re.findall(r'<a (id=\"[0-9]+">.*?)\n\n', url_out_raw, re.DOTALL)
        for i in albums_info:
            j = i.split('\n')[0]
            album = re.findall(r'<b>"(.*?)"</b>', j)[0]
            a_dict['albums'][album] = {}

            az_album_id = re.findall(r'id=\"([0-9]+)\">', j)
            a_dict['albums'][album]['az_album_id'] = az_album_id

            year = re.findall(r'</b> \(([0-9]+)\)</div>', j)[0]
            a_dict['albums'][album]['year'] = year

            lyrics_link = ['https://www.azlyrics.com'+item for item in re.findall(r'<a href="..(.*?)\" target', i)]
            a_dict['albums'][album]['lyrics_link'] = lyrics_link

        return a_dict

In [56]:
def search_to_link(query):
    
    query_encode = "http://search.azlyrics.com/search.php?"+urlencode({"q": query}) 
    r=requests.get(query_encode)
    if r.ok:
        link = re.findall(r'<a href=\"(.*?.html)\" target', r.text)[0]
        return link

In [2]:
def url_raw(pub_url):
    temp_file, _ = urlretrieve(pub_url)
    with open(temp_file) as file:
        return file.read()

In [94]:
def get_album_info(url_out_raw):
    header = re.findall(r'<h1><strong>(.*?) Lyrics</strong></h1>', url_out_raw)[0].strip()
    

    a_dict = {}
    a_dict['band_name'] = header
    try:
        related_artists_slab = re.findall(r'Related artists:<br>\n(.*?)\n\n',url_out_raw, re.DOTALL)[0].strip()
        related_artists = re.findall(r'\">(.*?)</a>', related_artists_slab)
        a_dict['related_artists'] = related_artists
    except:
        pass
    a_dict['albums'] = {}
    
    albums_info = re.findall(r'<a (id=\"[0-9]+">.*?)\n\n', url_out_raw, re.DOTALL)
    for i in albums_info:
        j = i.split('\n')[0]
        album = re.findall(r'<b>"(.*?)"</b>', j)[0]
        a_dict['albums'][album] = {}
        
        az_album_id = re.findall(r'id=\"([0-9]+)\">', j)[0]
        a_dict['albums'][album]['az_album_id'] = az_album_id

        year = re.findall(r'</b> \(([0-9]+)\)</div>', j)[0]
        a_dict['albums'][album]['year'] = year

        lyrics_link = ['https://www.azlyrics.com'+item for item in re.findall(r'<a href="..(.*?)\" target', i)]
        a_dict['albums'][album]['lyrics_link'] = lyrics_link
        
    return a_dict

In [5]:
url_link = 'https://www.azlyrics.com/p/poetsofthefall.html'
a = url_raw(url_link)
b = get_album_info(a)
b.keys()

dict_keys(['band_name', 'albums'])

In [157]:
def out_lyrics(pub_url):
    x = url_raw(pub_url)
    paragraphs = re.findall(r'<div>(.*?)</div>', str(x), re.DOTALL)[0]
    paragraphs = re.sub('<br>', '', paragraphs)
    paragraphs = re.sub(r'<!--.*?-->', '', paragraphs).strip()
    
    song_name = re.findall(r'SongName = \"(.*?)\";', str(x), re.DOTALL)[0]
    return song_name, paragraphs

In [168]:
def write_file(album_dictionary):
    if not os.path.exists('Lyrics'):
        os.mkdir('Lyrics')
    
    item = album_dictionary['band_name']
    path = 'Lyrics/%s'%item
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)

    with open(path+'/band_info.txt', 'w') as file:
        file.write('Band_name: '+ album_dictionary['band_name'])
        if 'related_artists' in album_dictionary.keys():
            file.write('Related Artists: '+ c['related_artists'])

    new_df = pd.DataFrame.from_dict(album_dictionary['albums'], orient='index')
    new_df.index.name = 'Album'
    stacked_titles = pd.DataFrame(pd.DataFrame(new_df['lyrics_link'].tolist(), index=new_df['az_album_id']).stack()).reset_index()
    stacked_titles.drop('level_1', inplace = True, axis = 1)
    stacked_titles.columns = ['az_album_id', 'lyrics_link']
    new_df_1 = new_df.reset_index().drop('lyrics_link', axis=1).merge(stacked_titles, on = 'az_album_id' ).set_index('az_album_id')
    new_df_1.to_csv(path+'/Albums.csv')
    
    new_df_1['folder_name'] = new_df_1['year'] + '_' + new_df_1['Album']
    new_df_1['lyrics_link'] = new_df_1['lyrics_link']
    new_df_1.reset_index(drop=True,inplace=True)

    for ind in range(len(new_df_1)):
        try:
            title, lyrics = out_lyrics(new_df_1['lyrics_link'][ind])
            new_path = re.sub('/', ':', path + '/' + new_df_1['folder_name'][ind])
            year = new_df_1['year'][ind]
            link = new_df_1['lyrics_link'][ind]
            album = new_df_1['Album'][ind]

            if os.path.exists(new_path):
                pass
            else:
                os.mkdir(new_path)
                
            with open(new_path+'/' + title + '.txt', 'w') as file:
                file.write('Song Name: ' + title + '\n' +
                          'Album: ' +  album + '|' + 'Year: ' + year + '\n' +
                          'link: ' + link + '-'*20 + '\n\n')
                file.write(lyrics)

        except Exception as e:
            print(e)

In [12]:
def azlyrics_links(query):
    '''
    DEPRECATED by get_album_info, ask Kevin about next page
    '''
    query_link = "http://search.azlyrics.com/search.php?"+urlencode({"q": query})
    
    r=requests.get(query_link)
    text = r.text
    link_info = re.findall(r'[0-9]+. <a href=\"(.*?)</b><br>', text)
    
    extracted_info = []
    
    for item in link_info:
        try:
            extr_info = (re.findall(r'.*html' , item),\
                    re.findall(r'<b>(.*?)</b>' , item),\
                    re.findall(r'<b>([^<>]*)\s*' , item)[1])
            
            extracted_info.append(extr_info)
            
        except:
            pass
    
    return extracted_info

In [13]:
azlyrics_links('The Beatles')

[(['https://www.azlyrics.com/lyrics/davidleeroth/thebeatlestune.html'],
  ['The Beatles Tune'],
  'David Lee Roth'),
 (['https://www.azlyrics.com/lyrics/joediffie/biggerthanthebeatles.html'],
  ['Bigger Than The Beatles'],
  'Joe Diffie'),
 (['https://www.azlyrics.com/lyrics/devendrabanhart/thebeatles.html'],
  ['The Beatles'],
  'Devendra Banhart'),
 (['https://www.azlyrics.com/lyrics/beatles/herecomesthesun.html'],
  ['Here Comes The Sun'],
  'The Beatles'),
 (['https://www.azlyrics.com/lyrics/beatles/acrosstheuniverse.html'],
  ['Across The Universe'],
  'The Beatles')]

In [166]:
a = search_to_link('poets of the fall')
b = url_raw(a)
c = get_album_info(b)
write_file(c)

[Errno 2] No such file or directory: 'Lyrics/Poets Of The Fall2006_Carnival Of Rust/All The Way / 4U.txt'
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response
Remote end closed co

In [96]:
c.keys()

dict_keys(['band_name', 'albums'])

In [158]:
azlyrics_url = "https://www.azlyrics.com/lyrics/poetsofthefall/carnivalofrust.html"
d = out_lyrics(azlyrics_url)
print(d[0]+'\n')
print(d[1])

Carnival Of Rust

D' you breathe the name of your saviour in your hour of need,
And taste the blame if the flavor should remind you of greed?
Of implication, insinuation and ill will, 'til you cannot lie still,
In all this turmoil, before red cape and foil come closing in for a kill

Come feed the rain
'cause I'm thirsty for your love dancing underneath the skies of lust
Yeah, feed the rain
'cause without your love my life ain't nothing but this carnival of rust

It's all a game, avoiding failure, when true colors will bleed
All in the name of misbehavior and the things we don't need
I lust for after no disaster can touch, touch us anymore
And more than ever, I hope to never fall, where enough is not the same it was before

Come feed the rain...
'cause I'm thirsty for your love dancing underneath the skies of lust
Yeah, feed the rain
'cause without your love my life ain't nothing but this carnival of rust
Yeah, feed the rain
'cause I'm thirsty for your love dancing underneath the skies